# **3374. First Letter Capitalization II**

**Table: user_content**
``` sql
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| content_id  | int     |
| content_text| varchar |
+-------------+---------+
```
content_id is the unique key for this table.

Each row contains a unique ID and the corresponding text content.

Write a solution to transform the text in the content_text column by applying the following rules:

Convert the first letter of each word to uppercase and the remaining letters to lowercase
Special handling for words containing special characters:

For words connected with a hyphen -, both parts should be capitalized (e.g., top-rated → Top-Rated)
All other formatting and spacing should remain unchanged

Return the result table that includes both the original content_text and the modified text following the above rules.

The result format is in the following example.

 

**Example:**

**Input:**

**user_content table:**
``` sql
+------------+---------------------------------+
| content_id | content_text                    |
+------------+---------------------------------+
| 1          | hello world of SQL              |
| 2          | the QUICK-brown fox             |
| 3          | modern-day DATA science         |
| 4          | web-based FRONT-end development |
+------------+---------------------------------+
```
**Output:**
``` sql
+------------+---------------------------------+---------------------------------+
| content_id | original_text                   | converted_text                  |
+------------+---------------------------------+---------------------------------+
| 1          | hello world of SQL              | Hello World Of Sql              |
| 2          | the QUICK-brown fox             | The Quick-Brown Fox             |
| 3          | modern-day DATA science         | Modern-Day Data Science         |
| 4          | web-based FRONT-end development | Web-Based Front-End Development |
+------------+---------------------------------+---------------------------------+
```
**Explanation:**

**For content_id = 1:**

- Each word's first letter is capitalized: "Hello World Of Sql"

**For content_id = 2:**

- Contains the hyphenated word "QUICK-brown" which becomes "Quick-Brown"
- Other words follow normal capitalization rules

**For content_id = 3:**

- Hyphenated word "modern-day" becomes "Modern-Day"
- "DATA" is converted to "Data"

**For content_id = 4:**
- Contains two hyphenated words: "web-based" → "Web-Based"
- And "FRONT-end" → "Front-End"

**Solution-1:**
``` sql
# Write your MySQL query statement below
WITH RECURSIVE cte AS (
    SELECT 
        content_id,
        content_text,
        TRIM(REGEXP_SUBSTR(content_text, '^[a-zA-Z-]+')) AS word,
        TRIM(REGEXP_REPLACE(content_text, '^[a-zA-Z-]+', '')) AS remain,
        1 AS rn
    FROM user_content
    UNION ALL
    SELECT 
        content_id,
        content_text,
        TRIM(REGEXP_SUBSTR(remain, '^[a-zA-Z-]+')) AS word,
        TRIM(REGEXP_REPLACE(remain, '^[a-zA-Z-]+', '')) AS remain,
        rn + 1
    FROM cte
    WHERE remain <> ''
)
,cte2 as(
SELECT content_id, content_text,CONCAT(UPPER(LEFT(word, 1)), LOWER(SUBSTRING(word, 2))) as word, rn
FROM cte
WHERE word IS NOT NULL)
, cte3 as(
SELECT content_id, content_text,
case when word like '%-%' then CONCAT(LEFT(word,LOCATE('-', word)),upper(SUBSTRING(word,LOCATE('-', word)+1,1)),
LOWER(SUBSTRING(word, LOCATE('-', word)+2))) else word end as word,rn
FROM cte2)

select content_id, content_text as original_text,
group_concat(word order by rn separator ' ') as converted_text
FROM cte3
group by content_id
ORDER BY content_id;

**Solution-2:**
``` sql
--WITH INITCAP(POSTGRESQL)
SELECT content_id, content_text original_text, initcap(content_text) converted_text FROM user_content;

**Solution-3:**
``` sql
--postgre sql
WITH CTE AS(
	SELECT content_id, content_text,  
    unnest(string_to_array(REPLACE(content_text,'-','  '), ' ')) AS new_text
	FROM user_content 
)
SELECT content_id, content_text AS original_text , 
REPLACE( STRING_AGG(UPPER(SUBSTR(new_text,1,1)) || LOWER(SUBSTR(new_text,2, LENGTH(new_text) - 1)), ' ') , ' - ', '-')  AS converted_text
FROM (
	SELECT content_id, content_text, CASE WHEN new_text = '' THEN '-' ELSE new_text END 
	FROM CTE
)sub
GROUP BY content_id, content_text
ORDER BY content_id

**Solution-3:**
``` sql
WITH split_text AS (
    SELECT
        content_id,
        content_text,
        -- ([^- ]+) matches one or more characters that are NOT hyphens or spaces (regular words)
        -- ([- ]+) matches one or more hyphens or spaces (special characters)
        -- 'g' flag for globally
        -- [1] gets the first element of the regexp_matches array
        (regexp_matches(content_text,
                        '(([^- ]+)|([- ]+))', 'g'))[1] AS tokens
    FROM user_content
)
SELECT
    content_id,
    content_text AS original_text,
    STRING_AGG(CONCAT(UPPER(SUBSTR(tokens, 1, 1)),
                      LOWER(SUBSTR(tokens, 2))),
               '') AS converted_text
FROM split_text
GROUP BY content_id, content_text -- needed since converted_text is aggregated
ORDER BY content_id;